# Анализ данных и проверка методов обработки данных 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from data_description import get_data_frame_settings
from processing import obj_to_cat


import DS_common.emissions as em
from DS_common.excel_writing import save_double_column_df
from sys import path

path.append(
    '/home/dranik/KFA/university/diplom_project/prog/data_processing/classification_power_predictor'
)

from classification_power_predictor\
import classification_power_predictor as cpp
from sklearn.metrics import roc_curve


import os
import os.path as os_path

# current project file name
cpfn = "diplom_project"

curr_dir = os.getcwd()
curr_dir = os_path.join(curr_dir[:curr_dir.find(cpfn)], cpfn)
data_storage = os_path.join(curr_dir, "data_storage")
input_filename = os_path.join(data_storage, "proc_1")
result_filename2 = os_path.join(data_storage, "proc_2")
result_filename3 = os_path.join(data_storage, "proc_3")

# Загрузка данных

In [4]:
data = pd.read_csv(
    input_filename + ".csv",
    index_col = 0
)

# Очистка от выбросов

**Мы не удаляем строки - а прописываем выбросам пропуск**
Почему мы так делаем?

1) Что-бы не удалять больше чем нужно;<br>
2) Что-бы оставлось побольше данных - нет смысла много удалять пока не оценена классифицирующая способность.

Отрицительный стаж работы - бред!

In [5]:
data.loc[
    data['Работа последнее место стаж лет'] > 0,
    'Работа последнее место стаж лет'
] = np.NaN

Отрицительный доход - бред. Да и на 100 рублей не выжить.

In [6]:
data.loc[
    data['Работа уровень дохода BYR'] > 100,
    'Работа уровень дохода BYR'
] = np.NaN

Неположитеьная сумма договора - бред

In [7]:
data.loc[
    data['Сумма договора'] < 0,
    'Сумма договора'
] = np.NaN

Ежедневный платеж

In [8]:
data.loc[
    data['Ежедневный платеж'] < 0,
    'Ежедневный платеж'
] = np.NaN

**Показались подозлительными размахи переменных**

1.Количество фактов просрочки по основному долгу;<br>
2.Максимальное количество дней просрочки;<br>
3.Общее количество запросов в КБ;<br>
4.Сумма кредитных лимитов;<br>
5.Сумма договора;<br>
6.ежедневный платеж.

In [9]:
em_research_list = [
    "Количество фактов просрочки по основному долгу",
    "Максимальное количество дней просрочки",
    "Общее количество запросов в КБ",
    "Сумма кредитных лимитов", "Сумма договора",
    "Ежедневный платеж"
]

Рассмотрим описание данных

In [10]:
def get_describes(df):
    result = pd.DataFrame()
    for col in df:
        result[col] = df[col].describe()
    return result

get_describes(data[em_research_list]).\
to_excel("result/emissions_research/emissions_describes_initial.xlsx")

Ну глядя на максимальное значений и 75-е персентили все становиться понятно - проведем удаление по правилу из Википедии.

In [11]:
def cut_emissioins(col):
    return col[em.get_selcond_emiss_25_75(col, cut_type = 'right')]

data[em_research_list] = \
data[em_research_list].apply(cut_emissioins)

In [12]:
#get_describes(data[em_research_list])
get_describes(data[em_research_list]).\
to_excel("result/emissions_research/emissions_describes_new.xlsx")

In [13]:
get_data_frame_settings(data).to_excel(
    "result/describe_predictors/4_emissions_drop.xlsx")

# Классифицирующая способность

In [14]:
def cl_abby(data, file_name):
    object_data = data.copy()
    cat_cols = object_data.columns[object_data.dtypes == 'category']
    object_data[cat_cols] = object_data[cat_cols].astype("O")

    my_cpp = cpp(
        object_data.drop('Y', axis = 1), 
        object_data['Y'].replace({0:"Нет дефолта", 1:"Дефолт"}))
    my_cpp.update_predictors()
    
    writer = pd.ExcelWriter(file_name)
    save_double_column_df(my_cpp.result_DF,writer)
    writer.close()
    return my_cpp.result_DF

result_DF = cl_abby(data, "result/classify_abby/1.xlsx")

## По незначимости KS статистики выборку покидают

In [15]:
result_DF = pd.read_excel(
    "result/classify_abby/1.xlsx",
    index_col = 0, header=[0,1]
)


drop_index = result_DF[
    result_DF[("Дефолт","KS_p_val")] > 0.01
].index
drop_index.tolist()

['Автомобиль год выпуска3',
 'Количество детей',
 'Отношение к банку',
 'Работа последнее место стаж лет',
 'Работа уровень дохода BYR',
 'Количество действующих договоров обеспечения',
 'Число авто',
 'Есть авто']

In [16]:
data.drop(drop_index, inplace = True, axis = 1)

cl_abby(data, "result/classify_abby/2.xlsx")

### Показатели оказавшиеся хуже связанных

In [17]:
drop_cols = [
    'Ежедневный платеж',
    'Автомобиль год выпуска'
]

data.drop(drop_cols, inplace = True, axis = 1)

Показатели "Столица", "Областной центр" и "Адрес проживания - Населенный пункт" хоть и созданы в отрыве от показателя "Тип насеренного пункта" очевидно с ним взаимосвязаны и заметно ему уступают, потому проиведем их удаление.

In [18]:
data.drop([
    "Столица", "Областной центр", 
     "Адрес проживания - Населенный пункт"
], axis = 1, inplace = True)

## Далее борьба с пропусками

Воинская служба много пропусков - но естесвенно понятно, что если мы укажем "невоеннообязанный" для всеж Ж у который пропуск то в 99% случаях окажемся правы.

In [19]:
data.loc[data['Пол'] == 'Ж', 'Воинская служба'] = "невоеннообязанный"

### Пропуски в числовых переменных

Найдем числовые переменные

In [20]:
numeric_cols = data.columns[
    (data.dtypes.apply(pd.api.types.is_numeric_dtype) & \
    np.invert(data.dtypes.apply(pd.api.types.is_bool_dtype)))
].drop(['Y', 'Дата рождения'])
numeric_cols

Index(['Автомобиль год выпуска1', 'Автомобиль год выпуска2',
       'Количество иждивенцев', 'Количество действующих кредитных договоров',
       'Количество запросов в КБ за последние 30 дней',
       'Количество фактов просрочки по основному долгу',
       'Максимальное количество дней просрочки',
       'Максимальный срок, на который заключался договор, в годах',
       'Общее количество запросов в КБ', 'Сумма кредитных лимитов',
       'Сумма договора', 'Срок кредита в днях'],
      dtype='object')

#### Пропуски в нули

Говорим что пропуски равны нулю

In [21]:
data[numeric_cols + '(пропуск => 0)'] = data[numeric_cols].apply(lambda x: x.fillna(0))

#### Перекодировка в бинарный

Если пропуск то false

In [22]:
data[numeric_cols + '(бинарный)'] \
= data[numeric_cols].apply(lambda x: x.isna())

#### Перекодировка на два класса по точке соотвесвтующей KS статистике и пропуски как отдельный уровень

In [23]:
def get_trio_col(col, y):
    
    # определение точки прелома
    norm_col = (col - np.min(col))/(np.max(col) - np.min(col))
    not_na_cond = np.invert(norm_col.isna())
    fpr, tpr, thresholds = roc_curve(y[not_na_cond], norm_col[not_na_cond])
    col_thres = col[thresholds[np.argmax(np.abs(fpr - tpr))] == norm_col].iloc[0]
    
    # распределение классов
    result = col.copy()
    result[not_na_cond & (col <= col_thres)] = "<=" + str(col_thres)
    result[not_na_cond & (col > col_thres)] = ">" + str(col_thres)
    
    return result

data[numeric_cols + '(разбивка по KS)'] = data[numeric_cols].apply(get_trio_col, y = data['Y'])

Сохраняем - только численные переменные и их свежие производные

In [24]:
numeric_and_derivative = numeric_cols.to_list() +\
(numeric_cols + '(пропуск => 0)').to_list() +\
(numeric_cols + '(бинарный)').to_list() +\
(numeric_cols + '(разбивка по KS)').to_list() + ['Y']

numeric_and_altern = cl_abby(
    data[
        numeric_and_derivative
    ],
    "result/classify_abby/3.xlsx"
)

numeric_and_altern.sort_index().to_excel("result/classify_abby/3.xlsx")

## Удаление лишних столбоцов

Постепенно из списка numeric_and_derivative будем выбрасывать столбцы что еще пригодяться

In [25]:
def save_col(col_name):
    numeric_and_derivative.remove(col_name)

Автомобили - хороший показатель, только очень много пропусков.

In [26]:
save_col('Автомобиль год выпуска1(разбивка по KS)')

Количесво дейсвующих кредитных договоров - лучше всего обнулять пропуски

In [27]:
save_col("Количество действующих кредитных договоров(пропуск => 0)")

И так далее по аналогии: количество иждивенцев не сохранить; сумма договора - по числам пропуск в ноль, но логически оказалось, что нулевая сумма договора - бред, там мало пропусков, потому оставим в изначальном виде.

In [28]:
save_col("Количество запросов в КБ за последние 30 дней(разбивка по KS)")
save_col("Количество фактов просрочки по основному долгу(пропуск => 0)")
save_col("Максимальное количество дней просрочки(пропуск => 0)")
save_col("Максимальный срок, на который заключался договор, в годах(разбивка по KS)")
save_col("Общее количество запросов в КБ(разбивка по KS)")
save_col("Срок кредита в днях(разбивка по KS)")
save_col("Сумма договора")
save_col("Сумма кредитных лимитов(пропуск => 0)")
save_col('Y')

In [29]:
data.drop(numeric_and_derivative, inplace = True, axis = 1)

Вернем производным столбцам названия их "родителей"

In [30]:

br_emptys = data.columns.str.find("(")
cond = (br_emptys != -1)

data.rename(columns = dict(zip(
    data.columns[cond],
    list(map(
        lambda x: x[0][:x[1]], zip(data.columns[cond], br_emptys[cond])
    ))
)), inplace = True)

## Удаляем то что осталось от пропусков

По тем, что остались численными надо удалить строки 

In [31]:
data.drop(data.loc[
    data[
        data.columns[data.dtypes.apply(pd.api.types.is_numeric_dtype)]
    ].apply(lambda x: any(x.isna()), axis = 1),:
].index, inplace = True)

Всем оставшимся делаем - нет данных

In [32]:
no_data_marker = "нет данных"
cat_cols = data.columns[data.dtypes == 'object']

data.fillna(no_data_marker, inplace = True)

# Отчеты на момент удаленный пропусков

In [33]:
class_emp_del = cl_abby(data, "result/classify_abby/4.xlsx")

In [34]:
get_data_frame_settings(data).to_excel('result/describe_predictors/5 emp del.xlsx')

In [35]:
data.to_csv(result_filename2 + ".csv")

# В результате моделирования было решено также ввести критерий по AUC

In [34]:
save_index = class_emp_del.index[
    class_emp_del.loc[:, ('Нет дефолта', 'AUC')] > 0.6
]

data[list(save_index) + ['Y']].to_csv(result_filename3 + ".csv")